In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '../../src'))

In [2]:
import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)

In [3]:
from data import load_target, load_covariates
from features.clustering import cluster_series

In [4]:
import torch
from darts.models import TFTModel
from darts.dataprocessing.transformers.scaler import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
import optuna
import numpy as np 
from sklearn.preprocessing import MaxAbsScaler
from darts.metrics import smape
from darts.dataprocessing.transformers.scaler import Scaler
from darts.utils.likelihood_models import GaussianLikelihood

In [5]:
 # Load Data
target_series = load_target('../../data/03_processed/on_forecourt_sessions.csv', group_cols='location_id',
                            time_col='date', value_cols='energy_delivered_kwh', static_cols=['num_evse'], freq='D')
covariates = load_covariates('../../data/03_processed/weather_ecad.csv', time_col='date',
                                value_cols=['temp_max', 'temp_min', 'sunshine', 'precip'], freq='D')

# Cluster Time Series
series = cluster_series(target_series, k=1, subset=None)[0]
VAL_LEN = int(len(series) * 0.7)
train, val = series[:VAL_LEN], series[VAL_LEN:]


# scale
scaler = Scaler(MaxAbsScaler())
train = scaler.fit_transform(train)
val = scaler.transform(val)

In [6]:
# define objective function
def objective(trial):
    # select input and output chunk lengths
    in_len = trial.suggest_int("in_len", 7, 64)
    out_len = trial.suggest_int("out_len", 1, in_len-1)

    # Other hyperparameters
    hidden_dim = trial.suggest_int("hidden_dim", 4, 32)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 5)
    num_attention_heads = trial.suggest_int("num_attention_heads", 1, 8)
    full_attention = trial.suggest_categorical("full_attention", [False, True])
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    include_day = trial.suggest_categorical("day", [False, True])

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=5, verbose=False)

    pl_trainer_kwargs = {"callbacks": [ early_stopper, pruner]}

    num_workers = 0

    # optionally also add the (scaled) year value as a past covariate
    if include_day:
        encoders = {"datetime_attribute": {"past": ["day"]},
                    "transformer": Scaler()}
    else:
        encoders = None

    # reproducibility
    torch.manual_seed(42)

    # build the TCN model
    model = TFTModel(
        hidden_size=hidden_dim, 
        lstm_layers=lstm_layers,
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        num_attention_heads=num_attention_heads,
        full_attention=full_attention,
        batch_size=32,
        n_epochs=100,
        add_encoders=encoders,
        nr_epochs_val_period=1,
        dropout=dropout,
        optimizer_kwargs={'lr': lr}, 
        random_state=0,
        model_name="tft_model",
        likelihood=GaussianLikelihood(),
        pl_trainer_kwargs=pl_trainer_kwargs,
        force_reset=True,
        save_checkpoints=True,
    )


    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])

    # train the model
    model.fit(
        series=train,
        val_series=model_val_set,
        num_loader_workers=num_workers,
    )

    # reload best model over course of training
    model = TFTModel.load_from_checkpoint("tft_model")

    # Evaluate how good it is on the validation set, using sMAPE
    preds = model.predict(series=train, n=VAL_LEN)
    smapes = smape(val, preds, n_jobs=-1, verbose=True)
    smape_val = np.mean(smapes)

    return smape_val if smape_val != np.nan else float("inf")


# for convenience, print some optimization trials information
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")


# optimize hyperparameters by minimizing the sMAPE on the validation set
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2023-05-23 11:38:28,878] A new study created in memory with name: no-name-2de6bdb9-de64-4440-9ecc-b45d05233abe
ValueError: TFTModel requires future covariates. The model applies multi-head attention queries on future inputs. Consider specifying a future encoder with `add_encoders` or setting `add_relative_index` to `True` at model creation (read TFT model docs for more information). These will automatically generate `future_covariates` from indexes.
[W 2023-05-23 11:38:29,152] Trial 0 failed with parameters: {'in_len': 38, 'out_len': 9, 'hidden_dim': 5, 'lstm_layers': 4, 'num_attention_heads': 7, 'full_attention': True, 'dropout': 0.0697843338441103, 'lr': 0.0007149933466435621, 'day': True} because of the following error: ValueError('TFTModel requires future covariates. The model applies multi-head attention queries on future inputs. Consider specifying a future encoder with `add_encoders` or setting `add_relative_index` to `True` at model creation (read TFT model docs for more inf

ValueError: TFTModel requires future covariates. The model applies multi-head attention queries on future inputs. Consider specifying a future encoder with `add_encoders` or setting `add_relative_index` to `True` at model creation (read TFT model docs for more information). These will automatically generate `future_covariates` from indexes.

In [ ]:
results = study.trials_dataframe()
results[results['value'] == results['value'].min()]